In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import folium

warnings.filterwarnings(action='ignore')
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.options.display.float_format="{:.5f}".format
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [14]:
data = pd.read_csv('data/data.csv')
#submission = pd.read_csv('data/submission.csv')
target = pd.read_csv('data/target.csv')
#student = pd.read_csv('data/인천_학령인구_추계.csv', encoding = 'cp949')

In [15]:
data.head()

수요지 id         X        Y  초등학령인구
0     C1 126.37150 37.46378       1
1     C2 126.47950 37.49578      46
2     C3 126.48150 37.49978      36
3     C4 126.48250 37.49578      81
4     C5 126.48350 37.49778      25

In [16]:
data.columns = ['수요지 id', '수요_X', '수요_Y', '초등학령인구']

In [17]:
data.head()

수요지 id      수요_X     수요_Y  초등학령인구
0     C1 126.37150 37.46378       1
1     C2 126.47950 37.49578      46
2     C3 126.48150 37.49978      36
3     C4 126.48250 37.49578      81
4     C5 126.48350 37.49778      25

In [18]:
data.columns = ['수요지 id', '수요_X', '수요_Y', '초등학령인구']

In [19]:
data.head()

수요지 id      수요_X     수요_Y  초등학령인구
0     C1 126.37150 37.46378       1
1     C2 126.47950 37.49578      46
2     C3 126.48150 37.49978      36
3     C4 126.48250 37.49578      81
4     C5 126.48350 37.49778      25

In [20]:
target.head()

학교 id         X        Y
0    F1 126.54392 37.46729
1    F2 126.56873 37.49630
2    F3 126.49277 37.48443
3    F4 126.62968 37.49578
4    F5 126.52829 37.45028

In [7]:
target.columns = ['학교 id', '공급_X', '공급_Y']

In [8]:
target.head()

학교 id      공급_X     공급_Y
0    F1 126.54392 37.46729
1    F2 126.56873 37.49630
2    F3 126.49277 37.48443
3    F4 126.62968 37.49578
4    F5 126.52829 37.45028

In [9]:
sum_df = pd.DataFrame()

# 반복문을 통해 df1 각 행에 df2를 붙이기
for _, row in target.iterrows():
    temp_df = pd.concat([pd.DataFrame(row).T] * len(data), ignore_index=True)
    temp_df = temp_df.join(data)
    sum_df = pd.concat([sum_df, temp_df])

sum_df

학교 id      공급_X     공급_Y 수요지 id      수요_X     수요_Y  초등학령인구
0      F1 126.54392 37.46729     C1 126.37150 37.46378       1
1      F1 126.54392 37.46729     C2 126.47950 37.49578      46
2      F1 126.54392 37.46729     C3 126.48150 37.49978      36
3      F1 126.54392 37.46729     C4 126.48250 37.49578      81
4      F1 126.54392 37.46729     C5 126.48350 37.49778      25
..    ...       ...      ...    ...       ...      ...     ...
306   F50 126.50114 37.51461   C307 126.65150 37.48278       2
307   F50 126.50114 37.51461   C308 126.65250 37.47278      12
308   F50 126.50114 37.51461   C309 126.65250 37.47978       1
309   F50 126.50114 37.51461   C310 126.65250 37.48078       1
310   F50 126.50114 37.51461   C311 126.65250 37.48178      11

[15550 rows x 7 columns]

In [10]:
sum_df = sum_df.drop('초등학령인구', axis = 1)

In [11]:
sum_df['위도_범위'] = sum_df.apply(lambda row: sorted([row['수요_Y'], row['공급_Y']]), axis=1)

In [12]:
sum_df['경도_범위'] = sum_df.apply(lambda row: sorted([row['수요_X'], row['공급_X']]), axis=1)

In [13]:
sum_df.head()

학교 id      공급_X     공급_Y 수요지 id      수요_X     수요_Y  \
0    F1 126.54392 37.46729     C1 126.37150 37.46378   
1    F1 126.54392 37.46729     C2 126.47950 37.49578   
2    F1 126.54392 37.46729     C3 126.48150 37.49978   
3    F1 126.54392 37.46729     C4 126.48250 37.49578   
4    F1 126.54392 37.46729     C5 126.48350 37.49778   

                             위도_범위                              경도_범위  
0  [37.4637824, 37.46729212308389]  [126.3715048, 126.54392107334832]  
1  [37.46729212308389, 37.4957824]  [126.4795048, 126.54392107334832]  
2  [37.46729212308389, 37.4997824]  [126.4815048, 126.54392107334832]  
3  [37.46729212308389, 37.4957824]  [126.4825048, 126.54392107334832]  
4  [37.46729212308389, 37.4977824]  [126.4835048, 126.54392107334832]

In [14]:
sum_df['유흥_개수'] = 0

In [15]:
sum_df

학교 id      공급_X     공급_Y 수요지 id      수요_X     수요_Y  \
0      F1 126.54392 37.46729     C1 126.37150 37.46378   
1      F1 126.54392 37.46729     C2 126.47950 37.49578   
2      F1 126.54392 37.46729     C3 126.48150 37.49978   
3      F1 126.54392 37.46729     C4 126.48250 37.49578   
4      F1 126.54392 37.46729     C5 126.48350 37.49778   
..    ...       ...      ...    ...       ...      ...   
306   F50 126.50114 37.51461   C307 126.65150 37.48278   
307   F50 126.50114 37.51461   C308 126.65250 37.47278   
308   F50 126.50114 37.51461   C309 126.65250 37.47978   
309   F50 126.50114 37.51461   C310 126.65250 37.48078   
310   F50 126.50114 37.51461   C311 126.65250 37.48178   

                                       위도_범위  \
0            [37.4637824, 37.46729212308389]   
1            [37.46729212308389, 37.4957824]   
2            [37.46729212308389, 37.4997824]   
3            [37.46729212308389, 37.4957824]   
4            [37.46729212308389, 37.4977824]   
..                                       ...   
306         [37.4827824, 37.514612976919096]   
307         [37.4727824, 37.514612976919096]   
308         [37.4797824, 37.514612976919096]   
309         [37.4807824, 37.514612976919096]   
310  [37.48178239999999, 37.514612976919096]   

                                 경도_범위  유흥_개수  
0    [126.3715048, 126.54392107334832]      0  
1    [126.4795048, 126.54392107334832]      0  
2    [126.4815048, 126.54392107334832]      0  
3    [126.4825048, 126.54392107334832]      0  
4    [126.4835048, 126.54392107334832]      0  
..                                 ...    ...  
306   [126.5011352571878, 126.6515048]      0  
307   [126.5011352571878, 126.6525048]      0  
308   [126.5011352571878, 126.6525048]      0  
309   [126.5011352571878, 126.6525048]      0  
310   [126.5011352571878, 126.6525048]      0  

[15550 rows x 9 columns]

In [16]:
sum_df.reset_index(drop=True, inplace=True)

In [17]:
sum_df

학교 id      공급_X     공급_Y 수요지 id      수요_X     수요_Y  \
0        F1 126.54392 37.46729     C1 126.37150 37.46378   
1        F1 126.54392 37.46729     C2 126.47950 37.49578   
2        F1 126.54392 37.46729     C3 126.48150 37.49978   
3        F1 126.54392 37.46729     C4 126.48250 37.49578   
4        F1 126.54392 37.46729     C5 126.48350 37.49778   
...     ...       ...      ...    ...       ...      ...   
15545   F50 126.50114 37.51461   C307 126.65150 37.48278   
15546   F50 126.50114 37.51461   C308 126.65250 37.47278   
15547   F50 126.50114 37.51461   C309 126.65250 37.47978   
15548   F50 126.50114 37.51461   C310 126.65250 37.48078   
15549   F50 126.50114 37.51461   C311 126.65250 37.48178   

                                         위도_범위  \
0              [37.4637824, 37.46729212308389]   
1              [37.46729212308389, 37.4957824]   
2              [37.46729212308389, 37.4997824]   
3              [37.46729212308389, 37.4957824]   
4              [37.46729212308389, 37.4977824]   
...                                        ...   
15545         [37.4827824, 37.514612976919096]   
15546         [37.4727824, 37.514612976919096]   
15547         [37.4797824, 37.514612976919096]   
15548         [37.4807824, 37.514612976919096]   
15549  [37.48178239999999, 37.514612976919096]   

                                   경도_범위  유흥_개수  
0      [126.3715048, 126.54392107334832]      0  
1      [126.4795048, 126.54392107334832]      0  
2      [126.4815048, 126.54392107334832]      0  
3      [126.4825048, 126.54392107334832]      0  
4      [126.4835048, 126.54392107334832]      0  
...                                  ...    ...  
15545   [126.5011352571878, 126.6515048]      0  
15546   [126.5011352571878, 126.6525048]      0  
15547   [126.5011352571878, 126.6525048]      0  
15548   [126.5011352571878, 126.6525048]      0  
15549   [126.5011352571878, 126.6525048]      0  

[15550 rows x 9 columns]

In [18]:
warning = pd.read_csv('data/인천유흥_위경도.csv', encoding = 'cp949')

In [19]:
warning.head()

Unnamed: 0       경도        위도                      도로명주소
0           0 37.47120 126.62348       인천광역시 중구 개항로 5 씨멘스클럽
1           1 37.47584 126.63071        인천광역시 중구 홍예문로98번길 9
2           2 37.46722 126.63573        인천광역시 중구 서해대로 462-3
3           3 37.47232 126.62370        인천광역시 중구 신포로27번길 32
4           4 37.47197 126.62342  인천광역시 중구 신포로23번길 32 수노래타운

In [20]:
# 에러 수정
warning.columns = ['Unnamed: 0', '위도', '경도', '도로명주소']

In [21]:
warning.head()

Unnamed: 0       위도        경도                      도로명주소
0           0 37.47120 126.62348       인천광역시 중구 개항로 5 씨멘스클럽
1           1 37.47584 126.63071        인천광역시 중구 홍예문로98번길 9
2           2 37.46722 126.63573        인천광역시 중구 서해대로 462-3
3           3 37.47232 126.62370        인천광역시 중구 신포로27번길 32
4           4 37.47197 126.62342  인천광역시 중구 신포로23번길 32 수노래타운

In [22]:
from tqdm import tqdm
import time

In [23]:
for i in tqdm(range(len(sum_df))):
    time.sleep(0.01)
    for j in range(len(warning)):
        if (sum_df.iloc[i, 6][0] <= warning.iloc[j,1] <= sum_df.iloc[i, 6][1]) and (sum_df.iloc[i, 7][0] <= warning.iloc[j,2] <= sum_df.iloc[i, 7][1]):
            sum_df.at[i, '유흥_개수'] += 1

100%|████████████████████████████████████████████████████████████████████████████| 15550/15550 [06:38<00:00, 38.99it/s]


In [24]:
sum_df

학교 id      공급_X     공급_Y 수요지 id      수요_X     수요_Y  \
0        F1 126.54392 37.46729     C1 126.37150 37.46378   
1        F1 126.54392 37.46729     C2 126.47950 37.49578   
2        F1 126.54392 37.46729     C3 126.48150 37.49978   
3        F1 126.54392 37.46729     C4 126.48250 37.49578   
4        F1 126.54392 37.46729     C5 126.48350 37.49778   
...     ...       ...      ...    ...       ...      ...   
15545   F50 126.50114 37.51461   C307 126.65150 37.48278   
15546   F50 126.50114 37.51461   C308 126.65250 37.47278   
15547   F50 126.50114 37.51461   C309 126.65250 37.47978   
15548   F50 126.50114 37.51461   C310 126.65250 37.48078   
15549   F50 126.50114 37.51461   C311 126.65250 37.48178   

                                         위도_범위  \
0              [37.4637824, 37.46729212308389]   
1              [37.46729212308389, 37.4957824]   
2              [37.46729212308389, 37.4997824]   
3              [37.46729212308389, 37.4957824]   
4              [37.46729212308389, 37.4977824]   
...                                        ...   
15545         [37.4827824, 37.514612976919096]   
15546         [37.4727824, 37.514612976919096]   
15547         [37.4797824, 37.514612976919096]   
15548         [37.4807824, 37.514612976919096]   
15549  [37.48178239999999, 37.514612976919096]   

                                   경도_범위  유흥_개수  
0      [126.3715048, 126.54392107334832]      0  
1      [126.4795048, 126.54392107334832]     24  
2      [126.4815048, 126.54392107334832]     24  
3      [126.4825048, 126.54392107334832]     24  
4      [126.4835048, 126.54392107334832]     24  
...                                  ...    ...  
15545   [126.5011352571878, 126.6515048]     10  
15546   [126.5011352571878, 126.6525048]     43  
15547   [126.5011352571878, 126.6525048]     10  
15548   [126.5011352571878, 126.6525048]     10  
15549   [126.5011352571878, 126.6525048]     10  

[15550 rows x 9 columns]

In [25]:
sum_df['유흥_개수'].value_counts()

0      3629
1      1069
10      598
8       587
2       570
       ... 
104       2
121       1
123       1
122       1
124       1
Name: 유흥_개수, Length: 131, dtype: int64

In [28]:
sum_df.to_csv('유흥업소_weight.csv', index=False)